# **Finetune LLMs to Predict Human Preference using Chatbot Arena conversations**
- This notebook contain a solution for the LLM Classification Finetuning on Kaggle

**Main objective:** Predict which responses users will prefer in a head-to-head battle between chatbots powered by large language models(LLMs).

The LLM Classification Finetuning competition challenges participants to predict user preferences between responses generated by different large language models (LLMs). The dataset comprises conversations from the Chatbot Arena, where users compare responses from two anonymous LLMs and select their preferred answer. The objective is to develop a model that accurately forecasts which response a user would favor in these head-to-head comparisons.





In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.sparse import hstack



# **1.Import the data**

In [ ]:
# Read the dataset
train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
submission = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")

In [ ]:
submission

In [ ]:
train.head(10)

In [ ]:
test

In [ ]:
test.info()

In [ ]:
print(f"train data shape: {train.shape}")
print(f"test data shape: {test.shape}")
print(f"sample_submission data shape: {submission.shape}")


**In the context of large language models (LLMs) and machine learning, a prompt refers to the input or instruction given to the model to generate a response or perform a task. It is the text or query that is provided to the model, which influences the generated output**

The prompt could be a question, statement, or instruction.
The response would be the model's output based on that prompt.


In [ ]:
print(test['prompt'][0])
print(test["response_a"][0])
print(test["response_b"][0])


**Lets generate a word cloud : A word cloud is a visual representation (image) of word data. In other words, it is a collection, or cluster, of words depicted in different sizes. The bigger and bolder the word appears, the more often it's mentioned within a given text and the more important it is**

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


wordcloud = WordCloud(width=800, height=800).generate(' '.join(test['prompt'].dropna()))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


# 2.Data Cleaning & Preparing


**After laoding the data we can clean it , ts the process of identifying and correcting or removing errors, inconsistencies, and inaccuracies in a dataset to ensure its quality and reliability.**

In [ ]:
#checking null or missing data
train.isnull()
test.isnull()
submission.isnull()

In [ ]:
#Check for duplicate data
print(test.duplicated().sum())
print(train.duplicated().sum())
print(submission.duplicated().sum())

In [ ]:
#check for duplicate id's
total_id = len(train["id"])
total_unique_id = len(train["id"].unique())

print("Total number of 'id' duplicates:")
print(total_id - total_unique_id)

# 3. Features and Labels

We need to processes the  dataset to determine the winner among three possible outcomes:
- `winner_model_a`
- `winner_model_b`
-  `winner_tie`.
  So we define a function, `which_winner`, to assign a numerical value to each outcome. If `winner_model_a` is 1, the function returns 0 to indicate Model A is the winner; if `winner_model_b` is 1, it returns 1 for Model B as the winner; and if `winner_tie` is 1, it returns 2 to indicate a tie.
--> This function is applied row by row to the , and the results are stored in a new column named `winner`.
--> Next, the `winner` column is converted into a string-based label, `winner_model`, where numerical values are mapped to meaningful descriptions: `0` is converted to "model a," `1` to "model b," and `2` to "winner tie."
  **This will help simplifying the representation of the target variable Y .**

In [ ]:
#barchart - model winner: winner model a, winner model b or winner tie
def which_winner(value):
    if  value["winner_model_a"] == 1:
         #winner model a
         value["winner_model_b"] = 0
         value["winner_tie"] = 0
         return 0
    elif value["winner_model_b"] == 1:
         #winner model b
         return 1
    elif value["winner_tie"] == 1:
         #winner tie
         return 2
    return None

train["winner"] = train.apply(which_winner, axis=1)

train["winner_model"] = train["winner"].astype(str)
train.loc[train["winner_model"] == "0", "winner_model"] = "model a"
train.loc[train["winner_model"] == "1", "winner_model"] = "model b"
train.loc[train["winner_model"] == "2", "winner_model"] = "winner tie"

result_model_winner = train["winner_model"].value_counts()
print("model winner:", result_model_winner)

print("-----")
print("Matplotlib barchart, model winner:")

barWidth = 0.75
plt.figure(figsize=(8, 7))
plt.bar(result_model_winner.index, result_model_winner.values, barWidth, color='b')
plt.ylabel('Counts', fontweight ='bold', fontsize = 15)
plt.xlabel('Model winner', fontweight ='bold', fontsize = 15)
plt.title('LLMs Value Counts - model winner', fontweight ='bold', fontsize = 15)
plt.show()

In [ ]:

train

In [ ]:
#transform text data into numerical form
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 150) #without max_features it crashes due to memory limit
vectorizer_prompt = vectorizer.fit_transform(train["prompt"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
vectorizer_response_a = vectorizer.fit_transform(train["response_a"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
vectorizer_response_b = vectorizer.fit_transform(train["response_b"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())



temp_prompt = vectorizer.transform(train["prompt"])
temp_response_a = vectorizer.fit_transform(train["response_a"])
temp_response_b = vectorizer.fit_transform(train["response_b"])

print("vectorizer representation for 'prompt':\n", temp_prompt.toarray())
print("vectorizer representation for 'response a':\n", temp_response_a.toarray())
print("vectorizer representation for 'response b':\n", temp_response_b.toarray())

print("Number of elements for the vectorizer representation for 'prompt':\n", temp_prompt.shape)
print("Number of elements for the vectorizer representation for 'response a':\n", temp_response_a.shape)
print("Number of elements for the vectorizer representation for 'response b':\n", temp_response_b.shape)

In [ ]:
#Using a simple feature and label selection
import numpy as np

# the prediction target
train_y = train["winner"].values
#choosing "features"
train_X = np.concatenate((temp_prompt.toarray(), temp_response_a.toarray(), temp_response_b.toarray()), axis=1)

print("Selecting The Prediction Target and Choosing Features Complete")

# 4. Model Selection and evaluating¶


**Logistic Regression**
Logistic Regression predicts the probability for each target class as requested .

In [ ]:
from sklearn.linear_model import LogisticRegression
from datetime import datetime

#start time to calculate the execution time
start_time = datetime.now()

print("Use logistic regression")
#Apply the Logistic Regression
model = LogisticRegression(max_iter=500, multi_class='multinomial', solver='saga')
model.fit(train_X, train_y)
#end time
end_time = datetime.now()
#calculate the execution time
execution_time = (end_time - start_time).total_seconds()
print(f"The execution time is : {execution_time} secondes")



Trying to evaluate the logistic regression model with basic Evaluation Metrics :
- Accuracy
- confusion matrix
- Clqssificqtion Report


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Split into validation and training data
train_X_train, train_X_val, train_y_train, train_y_val = train_test_split(
    train_X, train_y, test_size=0.2, random_state=42
)

# Record start time to calculate the execution time
start = datetime.now()

# Make predictions on the validation set
value_y_predict = model.predict(train_X_val)
print('Model predicted values:', value_y_predict)
print('True values:', train_y_val)

# Predicted probabilities
value_y_probabilities = model.predict_proba(train_X_val)
print('Model prediction probabilities (class-wise):\n', value_y_probabilities)

# Model accuracy
score = model.score(train_X_val, train_y_val)
print('The Model Accuracy Score:', score)

# Confusion matrix
conf_matrix = confusion_matrix(train_y_val, value_y_predict)
print("Confusion Matrix:\n", conf_matrix)

# Classification report
report = classification_report(train_y_val, value_y_predict)  # Arguments fixed
print("Classification Report:\n", report)



In [ ]:
from sklearn.metrics import log_loss

model_log_loss = log_loss(train_y_val, value_y_probabilities)

print('Model Log loss:', model_log_loss)

In [ ]:
vectorizer = TfidfVectorizer(max_features = 150) #without max_features it crashes due to memory limit
vectorizer_prompt = vectorizer.fit_transform(test["prompt"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
vectorizer_test_response_a = vectorizer.fit_transform(test["response_a"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
vectorizer_test_response_b = vectorizer.fit_transform(test["response_b"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())



temp_test_prompt = vectorizer.transform(test["prompt"])
temp_test_response_a = vectorizer.fit_transform(test["response_a"])
temp_test_response_b = vectorizer.fit_transform(test["response_b"])

print("vectorizer representation for 'prompt':\n", temp_test_prompt.toarray())
print("vectorizer representation for 'response a':\n", temp_test_response_a.toarray())
print("vectorizer representation for 'response b':\n", temp_test_response_b.toarray())

print("Number of elements for the vectorizer representation for 'prompt':\n", temp_test_prompt.shape)
print("Number of elements for the vectorizer representation for 'response a':\n", temp_test_response_a.shape)
print("Number of elements for the vectorizer representation for 'response b':\n", temp_test_response_b.shape)
test_X = np.concatenate((temp_test_prompt.toarray(), temp_test_response_a.toarray(), temp_test_response_b.toarray()), axis=1)
value_test_y_probabilities = model.predict_proba(test_X)
print('Model winner prediction, probability', value_test_y_probabilities) #winner model a | winner
output = pd.DataFrame({'id': test.id,
                        'winner_model_a': value_test_y_probabilities[:, 0],
                        'winner_model_b': value_test_y_probabilities[:, 1],
                        'winner_tie': value_test_y_probabilities[:, 2]})
output.to_csv('submission.csv', index=False)